# Comparing channels

- R: sequential
- G: complementary
- B: not match at all

In [1]:
import ppscore
import pandas
import os
import numpy
from scipy.stats import spearmanr
from IPython.core.display import display, HTML
from ete3 import Tree

/tmp/ipykernel_223072/499083497.py:6: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [2]:
tree_path = "../../../data/trees"
channels = ("full", "gray_r", "gray_g", "gray_b", "gray_mean")
datasets = ["orthologs_hemoglobin_beta", "orthologs_myoglobin", "orthologs_neuroglobin", "orthologs_cytoglobin"]

In [3]:
def read_and_compare(dataset, channel="full"):
    result_dict = {}
    path = f"{tree_path}/{channel}/{dataset}"
    control_df = pandas.read_csv(f"{path}/Control with Clustal Omega.csv", index_col=0)
    for file in os.listdir(path):
        if file.endswith(".csv"):
            basename = ".".join(file.split(".")[0:-1])
            try:
                df = pandas.read_csv(f"{path}/{file}", index_col=0)
                result_dict[basename] = round(numpy.sqrt(numpy.sum((control_df.values - df.values)**2)), 4)
            except FileNotFoundError:
                pass
    result_df = pandas.DataFrame(result_dict, index=[dataset])
    result_df.rename(columns={"Structural Similarity Index Measure": channel}, inplace=True)
    result_df.index.name = "dataset"
    return result_df

## Euclidean Distance from distance matrices

In [4]:
sum_dfs = []
for channel in channels:
    dfs = []
    for dataset in datasets:
        dfs.append(read_and_compare(dataset, channel))
    sum_dfs.append(pandas.concat(dfs))
pandas.concat(sum_dfs, axis=1).T.reset_index().drop_duplicates(["index"]).set_index("index").T

index,Global with Needleman-Wunsch,full,Control with Clustal Omega,Local with Smith–Waterman,gray_r,gray_g,gray_b,gray_mean
dataset,,,,,,,,
orthologs_hemoglobin_beta,0.9085,1.8705,0.0,0.8937,1.8100,1.8426,1.9614,1.1396
orthologs_myoglobin,0.9196,1.9899,0.0,0.9044,1.8112,1.9487,2.2140,1.1520
orthologs_neuroglobin,1.8043,5.6302,0.0,1.1514,5.6482,5.6883,5.4560,1.7182
orthologs_cytoglobin,6.3417,6.1544,0.0,3.1095,6.2565,6.2951,5.7378,3.2222


In [5]:
def read_and_linear_correlate(dataset, channel="full"):
    result_dict = {}
    path = f"{tree_path}/{channel}/{dataset}"
    control_df = pandas.read_csv(f"{path}/Control with Clustal Omega.csv", index_col=0)
    for file in os.listdir(path):
        if file.endswith(".csv"):
            basename = ".".join(file.split(".")[0:-1])
            try:
                df = pandas.read_csv(f"{path}/{file}", index_col=0)
                result_dict[basename] = round(spearmanr(control_df.values.flatten(), df.values.flatten())[0], 4)
            except FileNotFoundError:
                pass
    result_df = pandas.DataFrame(result_dict, index=[dataset])
    result_df.rename(columns={"Structural Similarity Index Measure": channel}, inplace=True)
    result_df.index.name = "dataset"
    return result_df

## Linear Correlation from distance matrices

In [6]:
sum_dfs = []
for channel in channels:
    dfs = []
    for dataset in datasets:
        dfs.append(read_and_linear_correlate(dataset, channel))
    sum_dfs.append(pandas.concat(dfs))
pandas.concat(sum_dfs, axis=1).T.reset_index().drop_duplicates(["index"]).set_index("index").T

/home/dmvieira/.virtualenvs/bioinfo2.0/lib/python3.10/site-packages/scipy/stats/_stats_py.py:4529: SpearmanRConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(SpearmanRConstantInputWarning())
/home/dmvieira/.virtualenvs/bioinfo2.0/lib/python3.10/site-packages/scipy/stats/_stats_py.py:4529: SpearmanRConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(SpearmanRConstantInputWarning())
/home/dmvieira/.virtualenvs/bioinfo2.0/lib/python3.10/site-packages/scipy/stats/_stats_py.py:4529: SpearmanRConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(SpearmanRConstantInputWarning())
/home/dmvieira/.virtualenvs/bioinfo2.0/lib/python3.10/site-packages/scipy/stats/_stats_py.py:4529: SpearmanRConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(SpearmanRConstantInputWarning

index,Global with Needleman-Wunsch,full,Control with Clustal Omega,Local with Smith–Waterman,gray_r,gray_g,gray_b,gray_mean
dataset,,,,,,,,
orthologs_hemoglobin_beta,0.9999,0.9984,1.0,0.9998,0.9977,0.9987,0.9922,NaN
orthologs_myoglobin,0.9999,0.9972,1.0,0.9998,0.9950,0.9977,0.9922,NaN
orthologs_neuroglobin,0.9999,0.9769,1.0,0.9992,0.9782,0.9804,0.9744,NaN
orthologs_cytoglobin,0.7775,0.4524,1.0,0.8623,0.4582,0.4586,0.4566,NaN


In [7]:
def read_and_correlate(dataset, channel="full"):
    result_dict = {}
    path = f"{tree_path}/{channel}/{dataset}"
    control_df = pandas.read_csv(f"{path}/Control with Clustal Omega.csv", index_col=0)
    for file in os.listdir(path):
        if file.endswith(".csv"):
            basename = ".".join(file.split(".")[0:-1])
            try:
                df = pandas.read_csv(f"{path}/{file}", index_col=0)
                result_dict[basename] = round(ppscore.score(
                    pandas.DataFrame({"x": control_df.values.flatten(), "y": df.values.flatten()}), "x", "y")["ppscore"], 4)
            except FileNotFoundError:
                pass
    result_df = pandas.DataFrame(result_dict, index=[dataset])
    result_df.rename(columns={"Structural Similarity Index Measure": channel}, inplace=True)
    result_df.index.name = "dataset"
    return result_df

## Predictive Power Score from distance matrices

In [8]:
sum_dfs = []
for channel in channels:
    dfs = []
    for dataset in datasets:
        dfs.append(read_and_correlate(dataset, channel))
    sum_dfs.append(pandas.concat(dfs))
pandas.concat(sum_dfs, axis=1).T.reset_index().drop_duplicates(["index"]).set_index("index").T

index,Global with Needleman-Wunsch,full,Control with Clustal Omega,Local with Smith–Waterman,gray_r,gray_g,gray_b,gray_mean
dataset,,,,,,,,
orthologs_hemoglobin_beta,0.9953,0.9812,0.9968,0.9944,0.9815,0.9851,0.9504,0.0
orthologs_myoglobin,0.9954,0.9699,0.9968,0.9925,0.9710,0.9715,0.9210,0.0
orthologs_neuroglobin,0.9876,0.9683,0.9964,0.9897,0.9672,0.9677,0.9600,0.0
orthologs_cytoglobin,0.8524,0.7331,0.9913,0.8584,0.7356,0.7354,0.7236,0.0


In [9]:
def read_and_tree_compare(dataset, channel="full", control="Control with Clustal Omega"):
    result_dict = {}
    path = f"{tree_path}/{channel}/{dataset}"
    control_tree = Tree(f"{path}/{control}.nw", format=1)
    for file in os.listdir(path):
        if file.endswith(".nw"):
            basename = ".".join(file.split(".")[0:-1])
            try:
                tree = Tree(f"{path}/{file}", format=1)
                result = control_tree.compare(tree, unrooted=True)
                result_dict[basename] = round(result["norm_rf"], 4)
            except FileNotFoundError:
                pass
    result_df = pandas.DataFrame(result_dict, index=[dataset])
    result_df.rename(columns={"Structural Similarity Index Measure": channel}, inplace=True)
    result_df.index.name = "dataset"
    return result_df

## Robinson-foulds distance from generated Trees

In [10]:
sum_dfs = []
for channel in channels:
    dfs = []
    for dataset in datasets:
        dfs.append(read_and_tree_compare(dataset, channel))
    sum_dfs.append(pandas.concat(dfs))
pandas.concat(sum_dfs, axis=1).T.reset_index().drop_duplicates(["index"]).set_index("index").T

index,Global with Needleman-Wunsch,Local with Smith–Waterman,full,Control with Clustal Omega,gray_r,gray_g,gray_b,gray_mean
dataset,,,,,,,,
orthologs_hemoglobin_beta,0.0000,0.0,0.0000,0.0,0.0000,0.0000,0.1667,1.0
orthologs_myoglobin,0.0000,0.0,0.0000,0.0,0.0833,0.0833,0.1667,1.0
orthologs_neuroglobin,0.0000,0.0,0.0833,0.0,0.0833,0.0833,0.0833,1.0
orthologs_cytoglobin,0.6667,0.5,0.5833,0.0,0.5833,0.5833,0.6667,1.0


In [11]:
sum_dfs = []
for channel in channels:
    dfs = []
    for dataset in datasets:
        dfs.append(read_and_tree_compare(dataset, channel, "Global with Needleman-Wunsch"))
    sum_dfs.append(pandas.concat(dfs))
pandas.concat(sum_dfs, axis=1).T.reset_index().drop_duplicates(["index"]).set_index("index").T

index,Global with Needleman-Wunsch,Local with Smith–Waterman,full,Control with Clustal Omega,gray_r,gray_g,gray_b,gray_mean
dataset,,,,,,,,
orthologs_hemoglobin_beta,0.0,0.00,0.0000,0.0000,0.0000,0.0000,0.1667,1.0
orthologs_myoglobin,0.0,0.00,0.0000,0.0000,0.0833,0.0833,0.1667,1.0
orthologs_neuroglobin,0.0,0.00,0.0833,0.0000,0.0833,0.0833,0.0833,1.0
orthologs_cytoglobin,0.0,0.25,0.6667,0.6667,0.6667,0.6667,0.5833,1.0


In [12]:
sum_dfs = []
for channel in channels:
    dfs = []
    for dataset in datasets:
        dfs.append(read_and_tree_compare(dataset, channel, "Local with Smith–Waterman"))
    sum_dfs.append(pandas.concat(dfs))
pandas.concat(sum_dfs, axis=1).T.reset_index().drop_duplicates(["index"]).set_index("index").T

index,Global with Needleman-Wunsch,Local with Smith–Waterman,full,Control with Clustal Omega,gray_r,gray_g,gray_b,gray_mean
dataset,,,,,,,,
orthologs_hemoglobin_beta,0.00,0.0,0.0000,0.0,0.0000,0.0000,0.1667,1.0
orthologs_myoglobin,0.00,0.0,0.0000,0.0,0.0833,0.0833,0.1667,1.0
orthologs_neuroglobin,0.00,0.0,0.0833,0.0,0.0833,0.0833,0.0833,1.0
orthologs_cytoglobin,0.25,0.0,0.5000,0.5,0.5000,0.5000,0.4167,1.0


In [13]:
def read_and_tree_compare_branches(dataset, channel="full"):
    result_dict = {}
    path = f"{tree_path}/{channel}/{dataset}"
    control_tree = Tree(f"{path}/Control with Clustal Omega.nw", format=1)
    for file in os.listdir(path):
        if file.endswith(".nw"):
            basename = ".".join(file.split(".")[0:-1])
            try:
                tree = Tree(f"{path}/{file}", format=1)
                result = control_tree.compare(tree, unrooted=True)
                result_dict[basename] = 1.0 - round(result["source_edges_in_ref"], 4)
            except FileNotFoundError:
                pass
    result_df = pandas.DataFrame(result_dict, index=[dataset])
    result_df.rename(columns={"Structural Similarity Index Measure": channel}, inplace=True)
    result_df.index.name = "dataset"
    return result_df

## Compatibility branch score from generated Trees

In [14]:
sum_dfs = []
for channel in channels:
    dfs = []
    for dataset in datasets:
        dfs.append(read_and_tree_compare_branches(dataset, channel))
    sum_dfs.append(pandas.concat(dfs))
pandas.concat(sum_dfs, axis=1).T.reset_index().drop_duplicates(["index"]).set_index("index").T

index,Global with Needleman-Wunsch,Local with Smith–Waterman,full,Control with Clustal Omega,gray_r,gray_g,gray_b,gray_mean
dataset,,,,,,,,
orthologs_hemoglobin_beta,0.0000,0.0000,0.0000,0.0,0.0000,0.0000,0.0769,0.4615
orthologs_myoglobin,0.0000,0.0000,0.0000,0.0,0.0385,0.0385,0.0769,0.4615
orthologs_neuroglobin,0.0000,0.0000,0.0385,0.0,0.0385,0.0385,0.0385,0.4615
orthologs_cytoglobin,0.3077,0.2308,0.2692,0.0,0.2692,0.2692,0.3077,0.4615


In [15]:
def read_and_compare_channels(dataset):
    result_dict = {}
    dfs = {}
    for channel in channels:
        result_dict[channel] = {}
        path = f"{tree_path}/{channel}/{dataset}"
        dfs[channel] = pandas.read_csv(f"{path}/Structural Similarity Index Measure.csv", index_col=0)
    for channel in channels:
        control_df = dfs[channel]
        for c, df in dfs.items():
            result_dict[channel][c] = round(numpy.sqrt(numpy.sum((control_df.values - df.values)**2)), 4)
    result_df = pandas.DataFrame.from_dict(result_dict, orient='index')
    result_df.index.name = "dataset"
    return result_df.style.background_gradient(axis=None, vmin=result_df.min().min(), vmax=result_df.max().max(), cmap="YlGnBu")

## Features Euclidean distance from distance matrices

In [16]:
for dataset in datasets:
    try:
        correlations = read_and_compare_channels(dataset)
    except FileNotFoundError:
        continue
    else:
        display(HTML(f"<center><h3>{dataset}</h3></center>"))
        display(correlations)        

,full,gray_r,gray_g,gray_b,gray_mean
dataset,,,,,
full,0.000000,0.094000,0.077200,0.167300,3.007600
gray_r,0.094000,0.000000,0.042500,0.259500,2.948200
gray_g,0.077200,0.042500,0.000000,0.243000,2.980800
gray_b,0.167300,0.259500,0.243000,0.000000,3.093200
gray_mean,3.007600,2.948200,2.980800,3.093200,0.000000


,full,gray_r,gray_g,gray_b,gray_mean
dataset,,,,,
full,0.000000,0.208400,0.085800,0.258700,3.140400
gray_r,0.208400,0.000000,0.179700,0.462700,2.961600
gray_g,0.085800,0.179700,0.000000,0.326700,3.098400
gray_b,0.258700,0.462700,0.326700,0.000000,3.362200
gray_mean,3.140400,2.961600,3.098400,3.362200,0.000000


,full,gray_r,gray_g,gray_b,gray_mean
dataset,,,,,
full,0.000000,0.169900,0.160200,0.382200,7.187300
gray_r,0.169900,0.000000,0.051100,0.541700,7.201800
gray_g,0.160200,0.051100,0.000000,0.540800,7.243400
gray_b,0.382200,0.541700,0.540800,0.000000,7.016900
gray_mean,7.187300,7.201800,7.243400,7.016900,0.000000


,full,gray_r,gray_g,gray_b,gray_mean
dataset,,,,,
full,0.000000,0.150700,0.177400,0.499000,8.902900
gray_r,0.150700,0.000000,0.053400,0.640200,9.004800
gray_g,0.177400,0.053400,0.000000,0.671400,9.043700
gray_b,0.499000,0.640200,0.671400,0.000000,8.465700
gray_mean,8.902900,9.004800,9.043700,8.465700,0.000000
